In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hyperspy.api import load
from pixstem.api import PixelatedSTEM
%matplotlib qt

from skimage.feature import match_template

file = load("input2.blo")
s = PixelatedSTEM(file.inav[0, 0])
image = np.array(s)

In [3]:
def distance(x1, y1, x2, y2):
    return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
def duplicates():
    global list
    global result
    for i in range(len(list) - 2):
        s = list[i]
        t = list[i + 1]
        if (distance(s[0], s[1], t[0], t[1])) < 5:
            if result[s[0]][s[1]] > result[t[0]][t[1]]:
                list.pop(i + 1)
            else:
                list.pop(i)
            i -= 1


template = image[67:77, 65:76]
result = match_template(image, template)
ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]

###############
list = []
for i in range(len(result)):
    for j in range(len(result[i])):
        if (result[i][j] > 0.92):
            list.append((i, j))
###############

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2)
ax3 = plt.subplot(1, 3, 3, sharex=ax2, sharey=ax2)

ax1.imshow(template, cmap=plt.cm.gray)
ax1.set_axis_off()
ax1.set_title('template')

ax2.imshow(image, cmap=plt.cm.gray)
ax2.set_axis_off()
ax2.set_title('image')
# highlight matched region
hcoin, wcoin = template.shape

###############
#duplicates()
for i in list:
    a, b = i
    circ = plt.Circle((b + 5, a + 4), 0.25, edgecolor='r', facecolor='none')
    ax2.add_patch(circ)

###############

ax3.imshow(result)
ax3.set_axis_off()
ax3.set_title('`match_template`\nresult')
# highlight matched region
ax3.autoscale(False)
ax3.plot(x, y, 'o', markeredgecolor='r', markerfacecolor='none', markersize=10)

plt.show()